In [1]:
!pip install -U openai
!pip install ortools

In [2]:
from ortools.linear_solver import pywraplp

# 创建一个 LP 求解器（GLOP）
solver = pywraplp.Solver.CreateSolver('GLOP')
print("OR-Tools 安装成功！")
print("GLOP 可用:", solver is not None)

OR-Tools 安装成功！
GLOP 可用: True


In [6]:
import os
import json 
from openai import OpenAI

client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
    api_key="sk-2e61a8b2400a4de78b32d12800ed91c0",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)



In [7]:
messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": """
我需要你先尝试理解一个两阶段分布鲁棒优化问题。我会先给你问题描述和数学模型，然后给你问题的数据，最后我需要你给出使用问题的最优解。

我考虑的是一个两阶段分布鲁棒的Hop-Constrained Network Design Problem。我先给你确定的Hop-Constrained Network Design Problem的描述和模型，等下我会继续给你两阶段分布鲁棒版本的问题。

\subsection{Hop-Constrained Network Design Problem} \label{subsec:hcndp} We consider an undirected graph $\mathcal{G} = (\mathcal{N}, \mathcal{E})$, where $\mathcal{N}$ is the set of nodes and $\mathcal{E}$ is the set of undirected edges $\{i,j\}$ connecting nodes $i$ and $j$. To model flow directionality, we replace each undirected edge $\{i,j\} \in \mathcal{E}$ with two directed arcs, $(i,j)$ and $(j,i)$. Additionally, we introduce an auxiliary arc set $\mathcal{A}$ to facilitate the formulation. For each pair of nodes $\langle p,q \rangle$, where $p$ is a source and $q$ is a destination, we define $d^{pq}$ as the demand from $p$ to $q$, $\mathcal{K}$ as the set of all commodities $\langle p,q \rangle$, and $h^{pq}$ as the hop parameter, representing the maximum allowed path length for commodity $\langle p,q \rangle$. The HCND problem involves three types of investments and operating costs: a fixed cost $f_{ij}$ that represents the cost of activating edge $\{i,j\}$, a variable cost $c_{ij}$ that represents the flow cost per unit on arc $(i,j)$, and a penalty cost $c^{pq}$ that incurs if the commodity $\langle p,q \rangle$ cannot be routed. The penalty cost $c^{pq}$ incurs either due to the absence of a feasible path from $p$ to $q$ or violation of the hop parameter $h^{pq}$ (i.e., the shortest available path exceed $h^{pq}$ hops). The HCND problem involves three types of decision variables: (i) $y_{ij} = 1$ if edge $\{i,j\}$ is selected in the design (0 otherwise), (ii) $x^{pq}_{ij} = 1$ if commodity $\langle p,q \rangle$ is routed via arc $(i,j)$ (0 otherwise), and (iii) $s^{pq} = 1$ if the commodity $\langle p,q \rangle$ is routed (0 otherwise). We then have the following arc-based formulation for the HCND problem: \begin{subeqnarray} \label{Formu:det-arc-based} &\min\limits_{\mathbf{y}, \mathbf{x}, \mathbf{s}} \ & \sum_{\{i,j\} \in \mathcal{E} } f_{ij} y_{ij} + \sum_{\langle p,q \rangle \in \mathcal{K}} \sum_{(i,j) \in \mathcal{A}} c_{ij} d^{pq} x^{pq}_{ij} + \sum_{\langle p,q \rangle \in \mathcal{K}} c^{pq} (1 - s^{pq}) \slabel{det-arc-based:obj} \\ &\text{s.t.} \ & \sum_{ j:(i,j) \in \mathcal{A} } x^{pq}_{ij} - \sum_{ j:(j,i) \in \mathcal{A} } x^{pq}_{ji} = \left\{ \begin{array}{rl} s^{pq} & \text{ if } i = p \\ -s^{pq} & \text{ if } i = q \\ 0 & \text{ if } i \neq p,q \end{array} \right., \ \forall \langle p,q \rangle \in \mathcal{K}, \slabel{det-arc-based:cons1}\\ && x^{pq}_{ij} + x^{pq}_{ji} \leq y_{ij}, \ \forall \langle p,q \rangle \in \mathcal{K}, \{i,j\} \in \mathcal{E}, \slabel{det-arc-based:cons2}\\ && \sum_{(i,j) \in \mathcal{A}} x^{pq}_{ij} \leq h^{pq}, \ \forall \langle p,q \rangle \in \mathcal{K}, \slabel{det-arc-based:cons3}\\ && y_{ij} = 0 \text{ or } 1, \ \forall \{i,j\} \in \mathcal{E}, \slabel{det-arc-based:cons4} \\ && x^{pq}_{ij} = 0 \text{ or } 1, \ \forall \langle p,q \rangle \in \mathcal{K}, (i,j) \in \mathcal{A}, \slabel{det-arc-based:cons5} \\ && s^{pq} = 0 \text{ or } 1, \ \forall \langle p,q \rangle \in \mathcal{K}. \slabel{det-arc-based:cons6} \end{subeqnarray} The objective function \eqref{det-arc-based:obj} minimizes the total cost comprising three components: the fixed design costs for activated edges, variable routing costs proportional to commodity flows, and penalty costs for the commodities that cannot be routed. The flow conservation constraints \eqref{det-arc-based:cons1} ensure each commodity either forms a valid path from its origin to destination (when $s^{pq}=1$) or incurs the corresponding penalty (when $s^{pq}=0$). Specifically, these constraints enforce unit flow outflow at the source node (when $s^{pq}=1$), unit flow inflow at the destination, and strict balance at all intermediate nodes. Constraints \eqref{det-arc-based:cons2} couple the routing decisions with design decisions by requiring that flow can only traverse edges selected in the design ($y_{ij}=1$), while permitting bi-directional usage of installed edges. Constraints \eqref{det-arc-based:cons3} represent the problem's distinctive feature, imposing that each commodity's path length cannot exceed its prescribed hop limit $h^{pq}$. This is implemented through a cardinality restriction on the used arcs in each commodity's path. The remaining constraints \eqref{det-arc-based:cons4}--\eqref{det-arc-based:cons6} enforce binary requirements on all decision variables, with $y_{ij}$ determining edge selection, $x_{ij}^{pq}$ controlling routing paths, and $s^{pq}$ indicating commodity satisfaction. While the arc-based formulation \eqref{Formu:det-arc-based} of the HCND problem provides an intuitive representation, extending to the TSDRHCND problem (in Section \ref{subsec:2-stage-dro}) creates computational challenges due to the presence of binary variables in both stages. To address this, we develop an alternative hop-indexed formulation where fixing the design variables $\mathbf{y}$ yields a subproblem with a exact linear programming relaxation. Building on the work of \cite{pirkul2003new} and \cite{botton2013benders}, we employ a staged representation of hops: for each commodity $\langle p, q\rangle$, we establish $h^{pq}$ stages, where each arc traversal corresponds to advancing one stage. In this framework, the first hop represents stage $1$ flow, the second hop represents stage $2$ flow, and so forth. Unlike formulation \eqref{Formu:det-arc-based} which uses binary variables $x^{pq}_{ij}$ to indicate routing decisions, our hop-indexed formulation introduces variables $x^{pq,l}_{ij}$ to track whether commodity $\langle p, q\rangle$ traverses arc $(i,j)$ specifically at stage $l$. % This temporal disaggregation enables more effective decomposition of the problem while maintaining the hop constraints. We then have the following hop-indexed formulation: \begin{subeqnarray} \label{Formu:det-hop-indexed} &\min\limits_{\mathbf{y}, \mathbf{x}, \mathbf{s}} \ & \sum_{\{i,j\} \in \mathcal{E} } f_{ij} y_{ij} + \sum_{\langle p, q\rangle \in \mathcal{K}} \sum_{(i,j) \in \mathcal{A}} \sum_{l=1}^{h^{pq}} c_{ij} d^{pq} x^{pq,l}_{ij} + \sum_{\langle p, q\rangle \in \mathcal{K}} c^{pq} (1 - s^{pq}) \slabel{det-hop-indexed:obj} \\ &\text{s.t.} \ & \sum_{j : (p,j) \in \mathcal{A}} x^{pq,1}_{pj} = s^{pq}, \ \forall \langle p, q\rangle \in \mathcal{K}, \slabel{Cons:det-hop-indexed-1} \\ && \sum_{i : (i,j) \in \mathcal{A}} x^{pq,l}_{ij} = \sum_{i : (j,i) \in \mathcal{A}} x^{pq,l+1}_{ji},\ \forall \langle p, q\rangle \in \mathcal{K}, j \in \mathcal{N} \setminus \{p,q\}, l \in [h^{pq}-1], \slabel{Cons:det-hop-indexed-2} \\ && \sum_{i : (i,q) \in \mathcal{A}} \sum_{l=1}^{h^{pq}} x^{pq,l}_{iq} = s^{pq}, \ \forall \langle p, q\rangle \in \mathcal{K}, \slabel{Cons:det-hop-indexed-3}\\ && x^{pq,l}_{ij} \leq y_{ij},\ x^{pq,l}_{ji} \leq y_{ij}, \ \forall \langle p, q\rangle \in \mathcal{K}, \{i,j\} \in \mathcal{E}, l \in [h^{pq}], \slabel{Cons:det-hop-indexed-4}\\ && y_{ij} = 0 \text{ or } 1, \ \forall \{i,j\} \in \mathcal{E}, \slabel{Cons:det-hop-indexed-5}\\ && x^{pq, l}_{ij} = 0 \text{ or } 1, \ \forall \langle p, q\rangle \in \mathcal{K}, (i,j) \in \mathcal{A}, l \in [h^{pq}] \slabel{Cons:det-hop-indexed-6}\\ && s^{pq} = 0 \text{ or } 1, \ \forall \langle p, q\rangle \in \mathcal{K}. \slabel{Cons:det-hop-indexed-7} \end{subeqnarray} The objective function \eqref{det-hop-indexed:obj} minimizes the total cost comprising fixed edge activation costs, multi-stage routing costs, and penalty costs for the commodities that cannot be routed. The formulation establishes a temporally-expanded network where constraints \eqref{Cons:det-hop-indexed-1} initiate exactly one unit of flow from the source node at stage $1$ when $s^{pq}=1$, while constraints \eqref{Cons:det-hop-indexed-2} maintain strict flow conservation at intermediate nodes by requiring stage $l$ inflows to be balanced by stage $l+1$ outflows, thereby creating a chain of flows where each hop corresponds to advancing to the next stage. Constraints \eqref{Cons:det-hop-indexed-3} ensure the flow is absorbed at the destination no later than stage $h^{pq}$. Constraints \eqref{Cons:det-hop-indexed-4} couple the flow variables with design decisions by preventing flow on unbuilt edges in any stage. Together, these constraints create a hop-indexed formulation that inherently satisfies both flow conservation and hop constraints through explicit stage-wise routing.

\subsection{Two-Stage Distributionally Robust Hop-Constrained Network Design Problem} \label{subsec:2-stage-dro} Building upon the hop-indexed formulation \eqref{Formu:det-hop-indexed} of the deterministic HCND problem, we now introduce the TSDRHCND problem considering edge failure uncertainty. This extension addresses the critical limitation of the deterministic HCND problem that assumes fixed network topology, whereas real-world networks face stochastic edge failures that disrupt flow routing. The two-stage structure naturally captures the sequential decision-making process: (i) first-stage decisions determine the network design before observing failures, while (ii) second-stage decisions adjust flow routing after observing the network design and failure realization. To handle the uncertainty, we construct a Wasserstein ambiguity set $\mathcal{P}$ containing plausible probability distributions of edge failures. We then minimizes the worst-case expected cost over $\mathcal{P}$, expressed as: \begin{align} \label{Formu:2sta-DRO} \min\limits_{\mathbf{y} \in \{0,1\}^{|\mathcal{E}|} } \ & \mathbf{f}^{\top} \mathbf{y} + \max\limits_{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}} \left[ Q(\mathbf{y}, \boldsymbol{\omega}) \right], \end{align} where $\boldsymbol{\omega}$ is the binary random vector representing uncertainty of the edge failures, $\mathbb{P}$ is the distribution of $\boldsymbol{\omega}$, and $Q(\mathbf{y}, \boldsymbol{\omega})$ represents the optimal second-stage cost under the network design $\mathbf{y}$ and edge failures $\boldsymbol{\omega}$. The DRO framework provides two key advantages over traditional stochastic programming and robust optimization: (i) it requires only partial distributional information (avoiding exact failure probabilities), and (ii) it guarantees robustness against distributional mis-specification through the ambiguity set. \subsubsection{Random Vector and Ambiguity Set} \label{subsubsec:rvset} In Problem \eqref{Formu:2sta-DRO}, $\boldsymbol{\omega} = (\omega_{ij}, \ \forall \{i,j\} \in \mathcal{E})^{\top}$ is a binary random vector to indicate whether edges $\{i,j\} \in \mathcal{E}$ fail, where $\omega_{ij} = 0$ indicates the edge fails (otherwise $1$). The support of $\boldsymbol{\omega}$ is defined to allow up to $M$ edges in the network to simultaneously be in a state of failure, as follows: \begin{align*} \Omega := \left\{ \boldsymbol{\omega} \in \{0,1\}^{|\mathcal{E}|} \ \middle| \ |\mathcal{E}| - \sum_{\{i,j\} \in \mathcal{E}} \boldsymbol{\omega}_{ij} \leq M \right\} = \{\boldsymbol{\omega}_1, \ldots, \boldsymbol{\omega}_{|\Omega|} \}. \end{align*} $\mathcal{P}$ is defined as a Wasserstein ambiguity set centered at an empirical distribution $\mathbb{P}^*$ with radius $\epsilon$, formally defined as: \begin{align*} \mathcal{P} := \left\{ \mathbb{P} \in \mathcal{M}(\Omega) \ | \ d_{\mathcal{W}}(\mathbb{P}, \mathbb{P}^*) \leq \epsilon \right\}, \end{align*} where $\mathcal{M}(\Omega)$ denotes the space of probability measures supported on $\Omega$, and $d_{\mathcal{W}}$ represents the Wasserstein metric. If $\mathbf{v}^* := (v_i^*, \ \forall i \in [|\Omega|])^{\top}$ are given probabilities corresponding to the empirical distribution $\mathbb{P}^*$ and $\mathbf{v} := (v_i, \ \forall i \in [|\Omega|])^{\top}$ are probabilities with respect to $\mathbb{P}$, the Wasserstein ambiguity set $\mathcal{P}$ can be reformulated as: \begin{subeqnarray} & \mathcal{V} := \big\{ \mathbf{v} \in \mathbb{R}^{|\Omega|} \ | \ & \sum_{i \in [|\Omega|]} \sum_{j \in [|\Omega|]} \|\boldsymbol{\omega}_i - \boldsymbol{\omega}_j \| k_{ij} \leq \epsilon, \slabel{v-1}\\ && \sum_{j \in [|\Omega|]} k_{ij} = v_i, \ \forall i \in [|\Omega|], \slabel{v-2} \\ && \sum_{i \in [|\Omega|]} k_{ij} = v_j^*, \ \forall j \in [|\Omega|], \slabel{v-3} \\ && \sum_{i \in [|\Omega|]} v_i = 1, \slabel{v-4} \\ && v_i \geq 0, \ \forall i \in [|\Omega|], \slabel{v-5} \\ && k_{ij} \geq 0, \ \forall i \in [|\Omega|], j \in [|\Omega|] \big\}. \slabel{v-6} \end{subeqnarray} \subsubsection{Second-Stage Problem} \label{subsubsec:secstagepro} In the second stage, given the design decision $\mathbf{y}$ and edge failure scenario $\boldsymbol{\omega}$, we have the following problem: \begin{subeqnarray} \label{Equ:sec-stage-problem} Q(\mathbf{y}, \boldsymbol{\omega}) = &\min\limits_{\mathbf{x}, \mathbf{s}} \ & \sum_{\langle p,q \rangle \in \mathcal{K}} \left( \sum_{(i,j) \in \mathcal{A}} \sum_{l=1}^{h^{pq}} c_{ij} d^{pq} x^{pq,l}_{ij} + c^{pq} (1 - s^{pq}) \right) \slabel{Equ:sec-stage-problem:obj} \\ &\text{s.t.} \ & \sum_{j : (p,j) \in \mathcal{A}} x^{pq,1}_{pj} = s^{pq}, \ \forall \langle p,q \rangle \in \mathcal{K}, \slabel{Equ:sec-stage-problem:cons1} \\ && \sum_{i : (i,j) \in \mathcal{A}} x^{pq,l}_{ij} = \sum_{i : (j,i) \in \mathcal{A}} x^{pq,l+1}_{ji},\ \forall \langle p,q \rangle \in \mathcal{K}, j \in \mathcal{N} \setminus \{p,q\}, l \in [h^{pq}-1], \slabel{Equ:sec-stage-problem:cons2} \\ && \sum_{i : (i,q) \in \mathcal{A}} \sum_{l=1}^{h^{pq}} x^{pq,l}_{iq} = s^{pq}, \ \forall \langle p,q \rangle \in \mathcal{K}, \slabel{Equ:sec-stage-problem:cons3} \\ && x^{pq, l}_{ij} \leq y_{ij} \boldsymbol{\omega}_{ij}, \ x^{pq, l}_{ji} \leq y_{ij} \boldsymbol{\omega}_{ij}, \ \forall \langle p,q \rangle \in \mathcal{K}, \{i,j\} \in \mathcal{E}, l \in [h^{pq}], \slabel{Equ:sec-stage-problem:cons4} \\ && x^{pq, l}_{ij} = 0 \text{ or } 1, \ \forall \langle p,q \rangle \in \mathcal{K}, (i,j) \in \mathcal{A}, l \in [h^{pq}], \slabel{Equ:sec-stage-problem:cons5} \\ && s^{pq} = 0 \text{ or } 1, \ \forall \langle p,q \rangle \in \mathcal{K}. \slabel{Equ:sec-stage-problem:cons6} \end{subeqnarray} The objective function \eqref{Equ:sec-stage-problem:obj} minimizes the second stage cost comprising routing costs and penalty costs. Constriants \eqref{Equ:sec-stage-problem:cons1}--\eqref{Equ:sec-stage-problem:cons3} and constraints \eqref{Equ:sec-stage-problem:cons5}--\eqref{Equ:sec-stage-problem:cons6} come from Problem \eqref{Formu:det-hop-indexed}. The constraints \eqref{Equ:sec-stage-problem:cons4} couple the flow variables with network design decisions $\mathbf{y}$ and edge failure scenario $\boldsymbol{\omega}$ by preventing flow on unbuilt or failed edges. Note that binary decision variables occur in the first stage (variable $\mathbf{y}$) and the second stage (variables $\mathbf{x}, \mathbf{s}$) of Problem \eqref{Formu:2sta-DRO}, which makes this TSDRHCND problem computationally challenging.

注意模型中的norm都是L2norm。

问题的数据如下
node_id : {0, 1, 2, 3, 4, 5} .
edge_id, node_i, node_j, f_ij, c_ij: 
0, 2, 0, 305.0, 305.0 / 6000.0;
1, 2, 1, 266.0, 266.0 / 6000.0;
2, 2, 3, 274.0, 274.0 / 6000.0;
3, 2, 5, 410.0, 410.0 / 6000.0; 
4, 0, 1, 63.0,  63.0 / 6000.0;
5, 0, 3, 176.0, 176.0 / 6000.0;
6, 0, 5, 284.0, 284.0 / 6000.0;
7, 0, 4, 460.0, 460.0 / 6000.0;
8, 1, 3, 173.0, 173.0 / 6000.0; 
9, 1, 5, 281.0, 281.0 / 6000.0; 
10, 1, 4, 457.0, 457.0 / 6000.0;
11, 3, 5, 132.0, 132.0 / 6000.0;
12, 3, 4, 304.0, 304.0 / 6000.0; 
13, 5, 4, 225.0, 225.0 / 6000.0.
commodity_id, source(p), sink(q), demand(d^{pq}), hop_limit: 
0, 0, 2, 1650, 2;
1, 0, 1, 3514, 2;
2, 0, 3, 1944, 2;
3, 0, 4, 12797, 2;
4, 1, 3, 3, 2; 
5, 1, 4, 22, 2; 
6, 2, 1, 5, 2;
7, 2, 3, 7, 2; 
8, 2, 4, 59, 2; 
9, 3, 1, 1, 2; 
10, 3, 2, 1, 2; 
11, 3, 4, 26, 2; 
12, 4, 1, 11, 2; 
13, 4, 2, 26, 2; 
14, 4, 3, 172, 2; 
15, 5, 0, 696, 2;
16, 5, 1, 5, 2; 
17, 5, 2, 35, 2;
18, 5, 3, 14, 2; 
19, 5, 4, 85, 2.

商品的penalty cost设置为$d^{pq} \sum c_{ij}$，即为14个边的flow成本之和 乘以 需求量。

M = 2. 
场景列表的顺序：边都是好的，坏一条边，坏两条边.
在经验分布中每个场景出现的次数：{0, 40, 32, 27, 38, 9, 20, 32, 46, 15, 27, 43, 12, 32, 20, 31, 3, 13, 0, 16, 0, 14, 0, 13, 0, 0, 4, 3, 2, 2, 0, 11, 0, 10, 0, 11, 0, 0, 3, 2, 27, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 6, 2, 2, 4, 0, 1, 1, 0, 17, 0, 13, 0, 0, 3, 3, 29, 0, 25, 0, 0, 15, 3, 0, 23, 1, 0, 0, 14, 15, 5, 0, 2, 2, 0, 0, 0, 0, 12, 28, 16, 12, 0, 14}.
 \epsilon = 0.8.

给定y，worse-case distribution下期望第二阶段成本即求解如下LP问题：
For any fixed $\mathbf{y}\in \{0,1\}^{|\mathcal{E}|}$, the inner maximization problem over $\mathbf{v}$ in Problem \eqref{Formu:2sta-DRO-refor} constitutes a linear program, which admits the following primal-dual pair:
% For any $\mathbf{y}\in \{0,1\}^{|\mathcal{E}|}$, the inner maximization problem associated with $\mathbf{v}$ of Problem \eqref{Formu:2sta-DRO-refor} is a linear program:
\begin{subeqnarray*}\label{Equ:vprimal}
& \max\limits_{\mathbf{v}} & \sum_{i \in [|\Omega|]}  v_i Q(\mathbf{y}, \boldsymbol{\omega}_i) \\   
& \text{s.t.} & \eqref{v-1}-\eqref{v-6} \\
= &\min\limits_{\eta, \mathbf{q}} &  \epsilon \eta + \sum_{j \in [|\Omega|]} v_j^* q_j  \\ 
& \text{s.t.} &  Q(\mathbf{y}, \omega_i) \leq \eta \|\boldsymbol{\omega}_i - \boldsymbol{\omega}_j \| + q_j,\ \forall i \in [|\Omega|], j \in [|\Omega|] \\
&& \eta \geq 0, 
\end{subeqnarray*}
where $\eta$ and $\mathbf{q}$ are dual variables.

在你回复的最后你应该给出如下答案：y_0, y_1, ..., y_13 分别代表每个edge安装的情况，0代表不安装，1代表安装
"""},
]

In [8]:
completion = client.chat.completions.create(
    model="qwen3-max",
    messages=messages,
)
tmp = completion.model_dump_json() 
tmp = json.loads(tmp) 
assistant = tmp["choices"][0]["message"]["content"]

In [10]:
# 然后我会给你问题的数据，最后你要根据你的理解尝试给出该问题的最优解


# 你只需要先猜一个一阶段的解，然后尝试求解上面问题，算第二阶段的成本和x，根据二阶段的计算获取信息，然后更新第一阶段的解，重复这个过程 给出你认为最好的解。

# 问题的数据如下
print(assistant)
messages.append({"role": "assistant", "content": assistant})
messages.append({"role": "user", "content": """
根据你给的y，我计算了所有的Q(y, w):
1519.37
11129.7
11129.7
11129.7
11129.7
11129.9
11129.8
11129.7
13182.5
11129.7
11129.7
11129.7
11131.5
11130.5
11132.7
1519.37
1519.37
1519.37
2677.51
8536.5
1921.45
3575
1519.37
1519.37
1519.37
1521.12
1520.12
1522.36
1519.37
1519.37
2677.51
8536.5
1921.45
3575
1519.37
1519.37
1519.37
1521.12
1520.12
1522.36
1519.37
2677.51
8536.5
1921.45
3575
1519.37
1519.37
1519.37
1521.12
1520.12
1522.36
2677.51
9694.8
3080.13
4733.14
2677.51
2677.51
2677.51
2679.26
2678.67
2680.5
8536.5
8938.58
10592.1
8536.5
8536.5
8536.5
8538.41
8537.25
8539.49
1921.45
3977.08
1921.45
1921.45
1921.45
1923.2
1922.69
1924.44
3575
3575
3575
3575
3576.75
3575.75
3577.99
1519.37
1519.37
1519.37
1521.12
1520.12
1522.36
1519.37
1519.37
1521.12
1520.12
1522.36
1519.37
1521.12
1520.12
1522.36
1521.12
1530.88
1537.4
1520.12
1527.39
1522.36，
以及worse-case distribution下期望第二阶段成本：
13968.5。 根据这些信息尝试改进你的解。
"""})

我需要解决这个两阶段分布鲁棒的Hop-Constrained Network Design Problem。让我逐步分析这个问题。

首先，让我理解问题的关键要素：

1. **网络结构**：6个节点(0-5)，14条边
2. **商品需求**：20个商品，每个都有源点、汇点、需求量和hop限制(都是2)
3. **不确定性**：最多M=2条边同时失效
4. **目标**：最小化第一阶段固定成本 + 最坏情况下的期望第二阶段成本

关键挑战是：
- 第一阶段决策变量y是二进制的（是否安装边）
- 第二阶段在给定y和失效场景ω下，需要求解路由问题
- 需要考虑Wasserstein模糊集中的最坏分布

由于这是一个复杂的混合整数规划问题，我需要考虑如何找到最优解。

**分析商品需求特点**：
- 大部分商品的hop限制为2，意味着路径最多经过2条边
- 有些商品需求量很大（如商品3: 0→4, 需求12797）
- 商品的penalty cost = d^{pq} × Σc_{ij}（所有14条边的flow成本之和）

**计算penalty cost**：
首先计算Σc_{ij} = (305+266+274+410+63+176+284+460+173+281+457+132+304+225)/6000
= 3810/6000 = 0.635

所以每个商品的penalty cost = d^{pq} × 0.635

**关键观察**：
1. 由于hop限制为2，很多商品可能无法直接连接，需要中间节点
2. 节点2是重要的中心节点（连接到0,1,3,5）
3. 高需求商品如0→4 (12797) 需要特别关注

**考虑最坏情况**：
- Wasserstein半径ε=0.8相对较大
- 经验分布中有些场景出现频率很高（如场景8出现46次）
- 最坏分布会将更多概率质量分配给高成本场景

**启发式分析**：
为了最小化总成本，我们需要：
1. 安装足够的边来满足高需求商品
2. 考虑边失效的鲁棒性
3. 平衡固定成本和路由/惩罚成本

**关键边识别**：
- 边7 (0-4): 直接连接高需求商品0→4，但固定成本460较高
- 边4 (0-1): 固定成本低(63)，连接多个商品
- 边11 (3-5): 固定成本低(132)，连接多个商品
- 节点2相

In [12]:
print(messages)
completion = client.chat.completions.create(
    model="qwen3-max",
    messages=messages,
)
tmp = completion.model_dump_json() 
tmp = json.loads(tmp) 
assistant = tmp["choices"][0]["message"]["content"]

[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': "\n我需要你先尝试理解一个两阶段分布鲁棒优化问题。我会先给你问题描述和数学模型，然后给你问题的数据，最后我需要你给出使用问题的最优解。\n\n我考虑的是一个两阶段分布鲁棒的Hop-Constrained Network Design Problem。我先给你确定的Hop-Constrained Network Design Problem的描述和模型，等下我会继续给你两阶段分布鲁棒版本的问题。\n\n\\subsection{Hop-Constrained Network Design Problem} \\label{subsec:hcndp} We consider an undirected graph $\\mathcal{G} = (\\mathcal{N}, \\mathcal{E})$, where $\\mathcal{N}$ is the set of nodes and $\\mathcal{E}$ is the set of undirected edges $\\{i,j\\}$ connecting nodes $i$ and $j$. To model flow directionality, we replace each undirected edge $\\{i,j\\} \\in \\mathcal{E}$ with two directed arcs, $(i,j)$ and $(j,i)$. Additionally, we introduce an auxiliary arc set $\\mathcal{A}$ to facilitate the formulation. For each pair of nodes $\\langle p,q \rangle$, where $p$ is a source and $q$ is a destination, we define $d^{pq}$ as the demand from $p$ to $q$, $\\mathcal{K}$ as the set of all commoditi

In [14]:
print(assistant)
messages.append({"role": "assistant", "content": assistant})
messages.append({"role": "user", "content": """
你新给出的解并不能改进成本。主要原因是你需要注意到，我给的Q(y,w)是按照边损坏的情况从全好到坏两条排序的。同时，你应该尝试搭建更多的边使得Q(y,w)更小来改进成本.
"""})

感谢您提供的Q(y, w)值和最坏情况下的期望第二阶段成本。让我分析这些结果并尝试改进解决方案。

从您提供的数据中，我可以看到：

1. **当前解的问题**：最坏情况下的期望第二阶段成本为13968.5，这非常高
2. **Q(y, w)值分析**：
   - 大部分场景的Q值在1519-3575之间（相对较低）
   - 但有很多场景的Q值高达8536.5、11129.7、13182.5（非常高）
   - 最高的是13182.5

这表明在某些失效场景下，我的当前解无法有效路由商品，导致高额惩罚成本。

**问题诊断**：
高Q值场景很可能对应于关键边（如边7: 0-4）失效的情况。当边7失效时，虽然我有替代路径（0-3-4和0-5-4），但如果这些替代路径中的边也同时失效，就会导致商品3（0→4，需求12797）无法路由，产生巨额惩罚成本。

**改进策略**：

1. **重新考虑边7（0-4）的必要性**：
   - 边7固定成本460较高
   - 如果不安装边7，而是专注于构建多条低成本替代路径，可能更鲁棒

2. **加强替代路径**：
   - 确保至少有3条独立的0→4路径，以应对最多2条边失效
   - 路径1: 0-1-4（需要边4和边10）
   - 路径2: 0-3-4（需要边5和边12）  
   - 路径3: 0-5-4（需要边6和边13）

3. **成本重新评估**：
   - 不安装边7（节省460固定成本）
   - 安装边10（1-4，成本457）来完善0-1-4路径
   - 这样总固定成本变化：-460 + 457 = -3（几乎不变）

4. **其他关键商品保障**：
   - 商品0（0→2，1650）：确保边0（2-0）安装
   - 商品1（0→1，3514）：边4（0-1）已安装
   - 商品2（0→3，1944）：边5（0-3）已安装
   - 商品15（5→0，696）：边6（0-5）已安装

**新的候选解**：
y_0 = 1 (2-0)
y_1 = 0 (2-1) 
y_2 = 0 (2-3)
y_3 = 0 (2-5)
y_4 = 1 (0-1)
y_5 = 1 (0-3)
y_6 = 1 (0-5)
y_7 = 0 (0-4) ← **改为不安装**
y_8 = 0 (1-3)
y_9 =

In [15]:
print(messages)
completion = client.chat.completions.create(
    model="qwen3-max",
    messages=messages,
)
tmp = completion.model_dump_json() 
tmp = json.loads(tmp) 
assistant = tmp["choices"][0]["message"]["content"]

[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': "\n我需要你先尝试理解一个两阶段分布鲁棒优化问题。我会先给你问题描述和数学模型，然后给你问题的数据，最后我需要你给出使用问题的最优解。\n\n我考虑的是一个两阶段分布鲁棒的Hop-Constrained Network Design Problem。我先给你确定的Hop-Constrained Network Design Problem的描述和模型，等下我会继续给你两阶段分布鲁棒版本的问题。\n\n\\subsection{Hop-Constrained Network Design Problem} \\label{subsec:hcndp} We consider an undirected graph $\\mathcal{G} = (\\mathcal{N}, \\mathcal{E})$, where $\\mathcal{N}$ is the set of nodes and $\\mathcal{E}$ is the set of undirected edges $\\{i,j\\}$ connecting nodes $i$ and $j$. To model flow directionality, we replace each undirected edge $\\{i,j\\} \\in \\mathcal{E}$ with two directed arcs, $(i,j)$ and $(j,i)$. Additionally, we introduce an auxiliary arc set $\\mathcal{A}$ to facilitate the formulation. For each pair of nodes $\\langle p,q \rangle$, where $p$ is a source and $q$ is a destination, we define $d^{pq}$ as the demand from $p$ to $q$, $\\mathcal{K}$ as the set of all commoditi

In [16]:
print(assistant)

您说得对，我需要重新分析这个问题。让我仔细思考：

从您提供的Q(y,w)值来看，前几个值是：
- 第1个：1519.37（所有边都好）
- 第2-8个：11129.7左右（坏1条边的情况）
- 第9个：13182.5（坏1条边的某个情况）

这说明当某一条特定的边失效时，成本急剧上升到11000+，这很可能是边7（0-4）失效导致的，因为商品3的需求量12797非常大。

**关键洞察**：
我的当前解只安装了8条边，但为了应对最多2条边同时失效的情况，我可能需要安装更多的边来提供足够的冗余。

**重新分析高成本场景**：
- Q=13182.5应该是最坏的单边失效场景
- 这很可能对应于边7（0-4）失效，而替代路径也不完整

**改进策略**：
1. **保留边7（0-4）**：直接路径对于超高需求商品3至关重要
2. **加强所有可能的替代路径**：确保即使边7失效，仍有完整的替代路径
3. **考虑安装更多边**：虽然固定成本增加，但可以显著降低第二阶段的期望成本

**具体分析商品3（0→4, 12797）的所有可能路径（hop≤2）**：
- 直接：0-4（边7）
- 通过1：0-1-4（需要边4和边10）
- 通过2：0-2-4（不可能，没有2-4边）
- 通过3：0-3-4（需要边5和边12）
- 通过5：0-5-4（需要边6和边13）

为了确保在任意2条边失效时商品3仍可路由，我需要：
- 安装边7（直接路径）
- 安装完整的至少两条替代路径

**成本效益分析**：
商品3的惩罚成本 = 12797 × 0.635 ≈ 8126
这意味着任何能避免惩罚的方案，只要额外固定成本 < 8126 都是值得的。

**最优策略**：
安装所有与高需求商品相关的边，包括：
- 所有连接节点0的边（边0,4,5,6,7）
- 所有连接节点4的边（边7,10,12,13）
- 关键中间连接（边11: 3-5）

让我考虑一个更全面的解决方案：

**候选解**：
y_0 = 1 (2-0) - 商品0,10,13,17需要
y_1 = 1 (2-1) - 提供2-1连接，商品6,12需要  
y_2 = 1 (2-3) - 提供2-3连接，商品7,10,14需要
y_3 = 1 (2-5) - 提供2-5连接，商品17需要
y_4 = 1 (0-1) -

In [12]:











































y_all = [1] * 14
q_val = solve_second_stage(y_all, scenarios[0])  # all good
print("Q(y, all good) =", q_val)

worst_exp = solve_worst_case_expectation(y_all)
print("Worst-case expected cost =", worst_exp)

KeyError: (0, 1, 0, 2)

In [5]:
import numpy as np
from ortools.linear_solver import pywraplp
from itertools import combinations
import math

def generate_scenarios(num_edges, M):
    """生成所有可能的故障场景"""
    scenarios = []
    
    # 所有边都好的场景
    scenarios.append([1] * num_edges)
    
    # 坏1条边的所有场景
    for e in range(num_edges):
        scenario = [1] * num_edges
        scenario[e] = 0
        scenarios.append(scenario)
    
    # 坏2条边的所有场景
    for e1, e2 in combinations(range(num_edges), 2):
        scenario = [1] * num_edges
        scenario[e1] = 0
        scenario[e2] = 0
        scenarios.append(scenario)
    
    return scenarios

def solve_second_stage(y, omega, nodes, edges, commodities, costs_flow, demands, hop_limits, penalties):
    """
    求解第二阶段问题 Q(y, omega)
    给定网络设计y和边故障情况omega，计算最优路由成本
    """
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None
    
    num_nodes = len(nodes)
    num_edges = len(edges)
    num_commodities = len(commodities)
    
    # 决策变量
    x = {}  # x[pq,l,ij] 表示商品<p,q>在阶段l通过弧(i,j)
    s = {}  # s[pq] 表示商品<p,q>是否被路由
    
    for k, (_, src, dst, demand, hop_limit) in enumerate(commodities):
        s[k] = solver.IntVar(0, 1, f's_{k}')
        
        for l in range(1, hop_limit + 1):
            for idx, (e, i, j, _, _) in enumerate(edges):
                # 弧(i,j)和(j,i)对应的变量
                if omega[idx] == 1 and y[idx] == 1:  # 边未故障且已选择
                    x[(k, l, idx, i, j)] = solver.IntVar(0, 1, f'x_{k}_{l}_{idx}_{i}_{j}')
                    x[(k, l, idx, j, i)] = solver.IntVar(0, 1, f'x_{k}_{l}_{idx}_{j}_{i}')
                else:
                    x[(k, l, idx, i, j)] = solver.IntVar(0, 0, f'x_{k}_{l}_{idx}_{i}_{j}')
                    x[(k, l, idx, j, i)] = solver.IntVar(0, 0, f'x_{k}_{l}_{idx}_{j}_{i}')
    
    # 约束1: 源节点在第1阶段发出单位流
    for k, (_, src, dst, demand, hop_limit) in enumerate(commodities):
        flow_out = solver.NumVar(0, solver.infinity(), f'flow_out_{k}')
        # flow_out.Clear()
        for idx, (e, i, j, _, _) in enumerate(edges):
            if i == src:
                if (k, 1, idx, i, j) in x:
                    flow_out += x[(k, 1, idx, i, j)]
            elif j == src:
                if (k, 1, idx, j, i) in x:
                    flow_out += x[(k, 1, idx, j, i)]
        solver.Add(flow_out == s[k])
    
    # 约束2: 中间节点流量守恒（不同阶段之间）
    for k, (_, src, dst, demand, hop_limit) in enumerate(commodities):
        for node in nodes:
            if node != src and node != dst:
                for l in range(1, hop_limit):  # l从1到hop_limit-1
                    flow_in = solver.NumVar(0, solver.infinity(), f'flow_in_{k}_{node}_{l}')
                    # flow_in.Clear()
                    flow_out = solver.NumVar(0, solver.infinity(), f'flow_out_{k}_{node}_{l+1}')
                    # flow_out.Clear()
                    
                    # 计算第l阶段流入节点的流
                    for idx, (e, i, j, _, _) in enumerate(edges):
                        if j == node and (k, l, idx, i, j) in x:
                            flow_in += x[(k, l, idx, i, j)]
                        elif i == node and (k, l, idx, j, i) in x:
                            flow_in += x[(k, l, idx, j, i)]
                    
                    # 计算第l+1阶段流出节点的流
                    for idx, (e, i, j, _, _) in enumerate(edges):
                        if i == node and (k, l+1, idx, i, j) in x:
                            flow_out += x[(k, l+1, idx, i, j)]
                        elif j == node and (k, l+1, idx, j, i) in x:
                            flow_out += x[(k, l+1, idx, j, i)]
                    
                    solver.Add(flow_in == flow_out)
    
    # 约束3: 目标节点接收流
    for k, (_, src, dst, demand, hop_limit) in enumerate(commodities):
        flow_in = solver.NumVar(0, solver.infinity(), f'flow_in_dst_{k}')
        # flow_in.Clear()
        
        for l in range(1, hop_limit + 1):
            for idx, (e, i, j, _, _) in enumerate(edges):
                if j == dst and (k, l, idx, i, j) in x:
                    flow_in += x[(k, l, idx, i, j)]
                elif i == dst and (k, l, idx, j, i) in x:
                    flow_in += x[(k, l, idx, j, i)]
        
        solver.Add(flow_in == s[k])
    
    # 约束4: 流量不能超过边容量（结合y和omega）
    for k, (_, src, dst, demand, hop_limit) in enumerate(commodities):
        for l in range(1, hop_limit + 1):
            for idx, (e, i, j, _, _) in enumerate(edges):
                if (k, l, idx, i, j) in x:
                    solver.Add(x[(k, l, idx, i, j)] <= y[idx] * omega[idx])
                if (k, l, idx, j, i) in x:
                    solver.Add(x[(k, l, idx, j, i)] <= y[idx] * omega[idx])
    
    # 目标函数
    total_cost = solver.NumVar(0, solver.infinity(), 'total_cost')
    # total_cost.Clear()
    
    # 添加路由成本
    for k, (_, src, dst, demand, hop_limit) in enumerate(commodities):
        for l in range(1, hop_limit + 1):
            for idx, (e, i, j, _, cost_flow) in enumerate(edges):
                if (k, l, idx, i, j) in x:
                    total_cost += cost_flow * demand * x[(k, l, idx, i, j)]
                if (k, l, idx, j, i) in x:
                    total_cost += cost_flow * demand * x[(k, l, idx, j, i)]
    
    # 添加惩罚成本
    for k, (_, src, dst, demand, hop_limit) in enumerate(commodities):
        total_cost += penalties[k] * (1 - s[k])
    
    solver.Minimize(total_cost)
    
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        return solver.Objective().Value()
    else:
        print(f"Second stage problem not solved optimally. Status: {status}")
        # 如果无可行解，返回很大的值作为惩罚
        return float('inf')

def solve_worst_case_distribution(y, scenarios, nodes, edges, commodities, costs_flow, demands, hop_limits, penalties, v_star, epsilon):
    """
    求解worst-case分布下的期望第二阶段成本
    即求解 max_v sum(v_i * Q(y, omega_i)) s.t. v in Wasserstein ambiguity set
    """
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None
    
    num_scenarios = len(scenarios)
    
    # 计算所有场景下的第二阶段成本
    Q_values = []
    for i, omega in enumerate(scenarios):
        q_val = solve_second_stage(y, omega, nodes, edges, commodities, costs_flow, demands, hop_limits, penalties)
        Q_values.append(q_val)
    
    # 决策变量
    v = [solver.NumVar(0, 1, f'v_{i}') for i in range(num_scenarios)]
    k = [[solver.NumVar(0, 1, f'k_{i}_{j}') for j in range(num_scenarios)] for i in range(num_scenarios)]
    
    # eta变量用于对偶形式
    eta = solver.NumVar(0, solver.infinity(), 'eta')
    
    # 目标函数：最大化期望成本
    objective = solver.NumVar(0, solver.infinity(), 'objective')
    # objective.Clear()
    for i in range(num_scenarios):
        objective += v[i] * Q_values[i]
    solver.Maximize(objective)
    
    # 约束1: Wasserstein距离约束
    wasserstein_sum = solver.NumVar(0, solver.infinity(), 'wasserstein_sum')
    # wasserstein_sum.Clear()
    for i in range(num_scenarios):
        for j in range(num_scenarios):
            # 计算L2范数 ||omega_i - omega_j||
            diff_norm = 0
            for e_idx in range(len(edges)):
                diff_norm += (scenarios[i][e_idx] - scenarios[j][e_idx]) ** 2
            diff_norm = math.sqrt(diff_norm)
            wasserstein_sum += diff_norm * k[i][j]
    solver.Add(wasserstein_sum <= epsilon)
    
    # 约束2: k_{ij}的行和等于v_i
    for i in range(num_scenarios):
        row_sum = solver.NumVar(0, solver.infinity(), f'row_sum_{i}')
        # row_sum.Clear()
        for j in range(num_scenarios):
            row_sum += k[i][j]
        solver.Add(row_sum == v[i])
    
    # 约束3: k_{ij}的列和等于v*_j
    for j in range(num_scenarios):
        col_sum = solver.NumVar(0, solver.infinity(), f'col_sum_{j}')
        # col_sum.Clear()
        for i in range(num_scenarios):
            col_sum += k[i][j]
        solver.Add(col_sum == v_star[j])
    
    # 约束4: v的概率和为1
    prob_sum = solver.NumVar(0, solver.infinity(), 'prob_sum')
    # prob_sum.Clear()
    for i in range(num_scenarios):
        prob_sum += v[i]
    solver.Add(prob_sum == 1)
    
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        return solver.Objective().Value(), v, Q_values
    else:
        print(f"Worst case distribution problem not solved optimally. Status: {status}")
        return float('inf'), None, None

# 数据准备
def prepare_data():
    # 节点
    nodes = [0, 1, 2, 3, 4, 5]
    
    # 边 (edge_id, node_i, node_j, f_ij, c_ij)
    edges = [
        (0, 2, 0, 305.0, 305.0 / 6000.0),
        (1, 2, 1, 266.0, 266.0 / 6000.0),
        (2, 2, 3, 274.0, 274.0 / 6000.0),
        (3, 2, 5, 410.0, 410.0 / 6000.0),
        (4, 0, 1, 63.0, 63.0 / 6000.0),
        (5, 0, 3, 176.0, 176.0 / 6000.0),
        (6, 0, 5, 284.0, 284.0 / 6000.0),
        (7, 0, 4, 460.0, 460.0 / 6000.0),
        (8, 1, 3, 173.0, 173.0 / 6000.0),
        (9, 1, 5, 281.0, 281.0 / 6000.0),
        (10, 1, 4, 457.0, 457.0 / 6000.0),
        (11, 3, 5, 132.0, 132.0 / 6000.0),
        (12, 3, 4, 304.0, 304.0 / 6000.0),
        (13, 5, 4, 225.0, 225.0 / 6000.0)
    ]
    
    # 商品 (commodity_id, source, sink, demand, hop_limit)
    commodities = [
        (0, 0, 2, 1650, 2), (1, 0, 1, 3514, 2), (2, 0, 3, 1944, 2), (3, 0, 4, 12797, 2),
        (4, 1, 3, 3, 2), (5, 1, 4, 22, 2), (6, 2, 1, 5, 2), (7, 2, 3, 7, 2),
        (8, 2, 4, 59, 2), (9, 3, 1, 1, 2), (10, 3, 2, 1, 2), (11, 3, 4, 26, 2),
        (12, 4, 1, 11, 2), (13, 4, 2, 26, 2), (14, 4, 3, 172, 2), (15, 5, 0, 696, 2),
        (16, 5, 1, 5, 2), (17, 5, 2, 35, 2), (18, 5, 3, 14, 2), (19, 5, 4, 85, 2)
    ]
    
    # 提取数据
    costs_fixed = [edge[3] for edge in edges]
    costs_flow = [edge[4] for edge in edges]
    demands = [comm[3] for comm in commodities]
    hop_limits = [comm[4] for comm in commodities]
    
    # 计算惩罚成本
    penalties = []
    for _, _, _, demand, _ in commodities:
        penalty = demand * sum(costs_flow)  # 使用所有边的flow成本之和
        penalties.append(penalty)
    
    # 生成场景
    M = 2
    scenarios = generate_scenarios(len(edges), M)
    
    # 经验分布概率 (基于给定的出现次数)
    occurrence_counts = [0, 40, 32, 27, 38, 9, 20, 32, 46, 15, 27, 43, 12, 32, 20, 31, 3, 13, 0, 16, 0, 14, 0, 13, 0, 0, 4, 3, 2, 2, 0, 11, 0, 10, 0, 11, 0, 0, 3, 2, 27, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 6, 2, 2, 4, 0, 1, 1, 0, 17, 0, 13, 0, 0, 3, 3, 29, 0, 25, 0, 0, 15, 3, 0, 23, 1, 0, 0, 14, 15, 5, 0, 2, 2, 0, 0, 0, 0, 12, 28, 16, 12, 0, 14]
    
    total_count = sum(occurrence_counts)
    v_star = [count / total_count for count in occurrence_counts]
    
    # 如果场景数量与v_star长度不匹配，扩展v_star
    if len(scenarios) != len(v_star):
        # 截断或填充v_star
        if len(v_star) > len(scenarios):
            v_star = v_star[:len(scenarios)]
        else:
            v_star.extend([0.0] * (len(scenarios) - len(v_star)))
    
    # 归一化v_star使其和为1
    total_prob = sum(v_star)
    if total_prob > 0:
        v_star = [p / total_prob for p in v_star]
    else:
        v_star = [1.0 / len(scenarios)] * len(scenarios)  # 均匀分布
    
    epsilon = 0.8
    
    return {
        'nodes': nodes,
        'edges': edges,
        'commodities': commodities,
        'costs_fixed': costs_fixed,
        'costs_flow': costs_flow,
        'demands': demands,
        'hop_limits': hop_limits,
        'penalties': penalties,
        'scenarios': scenarios,
        'v_star': v_star,
        'epsilon': epsilon,
        'M': M
    }

# 示例运行
if __name__ == "__main__":
    data = prepare_data()
    
    # 假设一个初始的y值（全选）
    y_initial = [0] * len(data['edges'])
    
    # 示例：求解特定场景下的第二阶段问题
    omega_example = data['scenarios'][0]  # 全部边都好的场景
    print("Scenario example:", omega_example)
    
    second_stage_cost = solve_second_stage(
        y_initial, omega_example, 
        data['nodes'], data['edges'], data['commodities'],
        data['costs_flow'], data['demands'], data['hop_limits'], data['penalties']
    )
    
    print(f"Second stage cost for scenario {omega_example}: {second_stage_cost}")
    
    # 求解worst-case分布问题
    worst_case_cost, v_solution, Q_values = solve_worst_case_distribution(
        y_initial, data['scenarios'],
        data['nodes'], data['edges'], data['commodities'],
        data['costs_flow'], data['demands'], data['hop_limits'], data['penalties'],
        data['v_star'], data['epsilon']
    )
    
    print(f"Worst case expected cost: {worst_case_cost}")
    print(f"Number of scenarios: {len(data['scenarios'])}")
    print(f"First few Q values: {Q_values[:5] if Q_values else 'None'}")

Scenario example: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Second stage cost for scenario [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]: -3.637978807091713e-12
Worst case distribution problem not solved optimally. Status: 2
Worst case expected cost: inf
Number of scenarios: 106
First few Q values: None
